In [ ]:
import os
from os.path import expanduser
import socket
import sys


# BELOW IS SYSTEM/PATH CONFIGURATION #
my_home = expanduser("~")
running_system = socket.gethostname()
# Configure machine's names here #
systemHLRN = 'blogin1'
systemLOCAL = 'Laurents-MacBook-Pro.local'
my_home = expanduser("~")
# Define local pass #
if running_system == systemLOCAL:
    print "computer = ", running_system
    wd_path = os.path.join(my_home,"Documents","projects", "awi-models")
elif running_system[0:1] == "b" or running_system == systemHLRN:
    print "computer = ", running_system
    wd_path = os.path.join(my_home,"awi-models")
else:
    print 'please configure your local machine : type socket.gethostname()'
print "working directory set to", wd_path
os.chdir(wd_path)
# IMPORT PY-FESOM MODULES
sys.path.append("codes/modules")
sys.path.append("codes/pyfesom-master/") # add pyfesom to search path
#########################################

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
from pylab import *
from load_mesh_data_new import *
import numpy as np
import datetime as dt
from PiecewiseNorm import PiecewiseNorm
from netCDF4 import Dataset
#import colormaps as cmaps
from matplotlib.colors import ListedColormap
import pyfesom as pf

In [ ]:
# PLOT CONFIGURATION #

# load colormaps #
cmap = plt.get_cmap('RdBu_r')
# figure export 
export_plot = True
# figure export definition
dpicnt=150
# choose simulation
runid='Arc12'
# choose date
first_year = 1990
last_year  = 2015
years      = np.arange(first_year,last_year+1,1)
months =np.linspace(0,11,12).astype(int)
# choose depth
get3d = True
# load the given biological tracer #
var_id1, var_id2 = 'tr06', 'tr15'
# export CSV
export_csv = True
########################

# AUTOMATIC DEFINITION OF PATHS
resultpath = '/scratch/usr/hbkvsk12/hlrn3_work2/results/'+runid+'/Oldfiles/'
savepath    = wd_path+'/figures/'+runid+'/'
meshpath    = wd_path+'/data/mesh/meshArc4.5/'
outputpath = '/scratch/usr/hbkoziel/Arc12/'

# Create figure directory if it does not exist
if os.path.exists(savepath) == False:
    try:
        os.mkdir(savepath)
        print ("Creation of the directory %s successfull" % savepath)
    except OSError:
        print ("Creation of the directory %s failed" % savepath)
else:
    print ("directory %s already existing" % savepath)

In [ ]:
mesh = fesom_mesh(meshpath, get3d = get3d)
#mesh = load_mesh(meshpath, usepickle=True,get3d=True)
mesh.n32 = mesh.n32-1
mesh.zlevs = -mesh.zlevs # depth is made negative

In [ ]:
#ncfile      = resultpath+runid+'.initial.mesh.diag.nc'
#f           = Dataset(ncfile, 'r')
#NodalArea = f.variables['cluster_area'][:]
#NodalVol3D    = f.variables['cluster_vol'][:]
#f.close()
mesh

In [ ]:
mesh.zlevs

In [ ]:
CHL  = np.zeros((len(years),len(mesh.x2)))
for mo in range(4,10):
    print mo
    for ye in range(0,len(years)):
     print years[ye]
     
     if years[ye] < 2000:
        ncfile  = resultpath+runid+'.'+str(years[ye])+'.oce.mean.nc'
        f       = Dataset(ncfile, 'r')    
        chl1     = f.variables[var_id1][mo,:]
        chl2     = f.variables[var_id2][mo,:]
        chl=chl1+chl2
        
     else:
        if mo ==4: 
         dayind = (range(45,59))
         month='APR'
        elif mo ==5: 
         dayind = (range(60,74))
         month='MAY'
        elif mo ==6: 
         dayind = (range(75,89))
         month='JUN'
        elif mo ==7: 
         dayind = (range(90,105))
         month='JUL'
        elif mo ==8:
         dayind = (range(106,120))
         month='AUG'
        elif mo ==9: 
         dayind = (range(121,135))
         month='SEP'
            
        ncfile  = resultpath+runid+'.'+str(years[ye])+'.oce.mean.nc'
        f       = Dataset(ncfile, 'r')    
        chl1     = f.variables[var_id1][dayind,:].mean(axis=0)
        chl2     = f.variables[var_id2][dayind,:].mean(axis=0)
        chl=chl1+chl2
        
     depth_list = [    0.,   -10.,   -20.,   -30.,   -40.,   -50.,   -60.,   -70.,
         -80.,   -90.,  -100.,  -115.,  -135.,  -160.,  -190.,  -230.,
        -280.,  -340.,  -410.,  -490.]
     level_data  = np.zeros((len(mesh.x2),len(depth_list)))
     for i in range(len(depth_list)):

        print 'Load data at '+str(depth_list[i])+'m'
        level_data1, elem_no_nan1 = pf.get_data(chl,mesh,depth_list[i])
        level_data1=np.nan_to_num(level_data1)
        #nan_array = np.isnan(level_data1)
        #level_data1[nan_array]=0
        level_data[:,i]  = level_data1
        
     data_int=np.trapz(-level_data,depth_list,axis=1)
     CHL[ye,:]=data_int
    data2=CHL.mean(axis=1)
    print 'Number of nans in tracer: ',np.count_nonzero(np.isnan(data2))
    print 'Number of inf in tracer: ',np.count_nonzero(np.isinf(data2))
    print 'Mean of surface: ',np.mean(data2[0:len(mesh.x2)])
    print 'Max and min: ',np.max(data2),np.min(data2)

    if export_csv == True:
        np.savetxt(outputpath+'CHLint_'+month+'_'+str(first_year)+'_'+str(last_year)+'_trend.csv', CHL, delimiter=";")
        np.savetxt(outputpath+'CHLint_'+month+'_'+str(first_year)+'_'+str(last_year)+'_mean.csv', data2, delimiter=";")
        print 'exporting done ...'